# Import Libraries

In [3]:
%pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from kafka import KafkaConsumer
import sqlite3
import json

# Criação de uma tabela no SQLite

In [5]:
conn = sqlite3.connect('data.sql')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS league (
    id INTEGER PRIMARY KEY,
    accountId INTEGER,
    puuid TEXT,
    name TEXT,
    profileIconId INTEGER,
    revisionDate DATE,
    summonerLevel INTEGER);
''')
conn.commit()

print("Tabela 'league' criada com sucesso!")
cursor.close()
conn.close()

Tabela 'league' criada com sucesso!


In [6]:
conn = sqlite3.connect('data.sql')
cursor = conn.cursor()
cursor.execute('DELETE FROM league')
conn.commit()

print("Todos os dados da tabela 'league' foram deletados!")
cursor.close()
conn.close()

Todos os dados da tabela 'league' foram deletados!


# Leitura e inserção de dados no SQLite

In [ ]:
# Configurações do Kafka Consumer
consumer = KafkaConsumer('atividade_kafka', bootstrap_servers='localhost:9092')

# Conexão ao SQLite
conn = sqlite3.connect('data.sql')
cursor = conn.cursor()

def is_numeric(value):
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False

# Função para tratar os dados de cada coluna
def process_data(data):
    # Verifica e trata cada coluna
    data['id'] = data.get('id', None)
    data['accountId'] = bool(data.get('adult', False))
    data['puuid'] = data.get('backdrop_path', None)
    data['name'] = data.get('belongs_to_collection', None)
    data['profileIconId'] = data.get('budget', None)
    data['revisionDate'] = data.get('homepage', None)
    data['summonerLevel'] = data.get('imdb_id', None)

    # Adicione verificações e tratamentos adicionais para outras colunas, se necessário

    return data

# Função callback para processar mensagens do Kafka
def kafka_callback(message):
    try:
        if message.value:
            data = json.loads(message.value)

            if isinstance(data, dict):
                # Processa os dados antes da inserção
                data = process_data(data)

                cursor.execute('''
                INSERT INTO movies (
                    id, adult, backdrop_path, belongs_to_collection, budget, homepage,
                    imdb_id, original_language, original_title, overview, popularity,
                    poster_path, video, vote_average, vote_count, name, url
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    data['id'], data['adult'], data['backdrop_path'], 
                    data['belongs_to_collection'], data['budget'], data['homepage'],
                    data['imdb_id'], data['original_language'], data['original_title'], 
                    data['overview'], data['popularity'], data['poster_path'],
                    data['video'], data['vote_average'], data['vote_count'], 
                    data['name'], data['url']
                ))
                conn.commit()

                print(f"Dados do filme {data['name']} foram inseridos no SQLite.")
            else:
                print("Erro: Mensagem não está em formato JSON válido. Ignorando...")

        print("Mensagem do Kafka ouvida: ", message.value)

    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar a mensagem JSON: {e}")
    except Exception as e:
        print(f"Erro ao inserir no SQLite: {e}")

# Loop para consumir mensagens do Kafka
for message in consumer:
    kafka_callback(message)

# Fechando a conexão do SQLite
conn.close()